In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from bing_image_downloader import downloader

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

import os, shutil
from pathlib import Path
import random

from duckduckgo_search import DDGS
import httpx
import requests, os, random, shutil
from tqdm import tqdm
from pathlib import Path


In [ ]:
"""Download the data images"""

#returns this
# "title": "File:The Sun by the Atmospheric Imaging Assembly of NASA's Solar ...",
# "image": "https://upload.wikimedia.org/wikipedia/commons/b/b4/The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA's_Solar_Dynamics_Observatory_-_20100819.jpg",
# "thumbnail": "https://tse4.mm.bing.net/th?id=OIP.lNgpqGl16U0ft3rS8TdFcgEsEe&pid=Api",
# "url": "https://en.wikipedia.org/wiki/File:The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA's_Solar_Dynamics_Observatory_-_20100819.jpg",
# "height": 3860,
# "width": 4044,
# "source": "Bing",

search_query = "Barack Obama"

results = DDGS().images(
    keywords=search_query,
    region="wt-wt",
    safesearch="off",
    size=None,
    color=None,
    type_image=None,
    layout=None,
    license_image=None,
    max_results=200,
)

#Using httpx, download each image from the results.image field of duckduckgo
valid_exts = ('png', 'jpg', 'jpeg')
downloaded = 0
for i, result in enumerate(results):
    try:
        image_url = result['image']
        ext = image_url.split('.')[-1].split('?')[0].lower()  # get extension and lowercase it
        
        if ext not in valid_exts:
            # Skip if not an allowed image type
            continue
        
        response = httpx.get(image_url, timeout=10)
        if response.status_code != 200:
            continue
        
        filename = f'img{i}.{ext}'
        with open(f'./images/{filename}', 'wb') as f:
            f.write(response.content)
        
        downloaded += 1
        print(f'Downloaded: {downloaded}/{len(results)}', end='\r')  # progress on one line
        
    except Exception:
        continue

print('\n✅ Download done.')

#Once the images are downloaded half will get moved to train and half to test
src_folder = Path("images")
train_folder = Path(f'data/train/{search_query}')
test_folder = Path(f'data/test/{search_query}')
train_folder.mkdir(parents=True, exist_ok=True)
test_folder.mkdir(parents=True, exist_ok=True)

all_images = list(src_folder.glob("*"))
random.shuffle(all_images)

# Part you move the images to data folders
amountInTrain = 100 #Amount you put in train and rest in test
for i, img_path in enumerate(all_images):
    if i < amountInTrain:
        shutil.move(str(img_path), train_folder / img_path.name)
    else:
        shutil.move(str(img_path), test_folder / img_path.name)


Downloaded: 170/200
✅ Download done.


In [32]:
#Transforms the images to the same, correct size

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])


In [33]:
train_data = torchvision.datasets.CIFAR10

In [34]:
# 2. Load your custom dataset using ImageFolder
train_data = ImageFolder(root="data/train", transform=transform)
test_data = ImageFolder(root="data/test", transform=transform)

# 3. Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [36]:
class_names = ['Barack Obama']

In [38]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [39]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [40]:
for epoch in range(30):
    print(f'Training epoch  {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data 

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch  0...
Loss: 2.3903
Training epoch  1...
Loss: 2.3710
Training epoch  2...
Loss: 2.3411
Training epoch  3...
Loss: 2.3014
Training epoch  4...
Loss: 2.2588
Training epoch  5...
Loss: 2.2123
Training epoch  6...
Loss: 2.1624
Training epoch  7...
Loss: 2.1112
Training epoch  8...
Loss: 2.0609
Training epoch  9...
Loss: 2.0070
Training epoch  10...
Loss: 1.9519
Training epoch  11...
Loss: 1.8910
Training epoch  12...
Loss: 1.8248
Training epoch  13...
Loss: 1.7556
Training epoch  14...
Loss: 1.6785
Training epoch  15...
Loss: 1.5839
Training epoch  16...
Loss: 1.4707
Training epoch  17...
Loss: 1.3322
Training epoch  18...
Loss: 1.1344
Training epoch  19...
Loss: 0.8693
Training epoch  20...
Loss: 0.5178
Training epoch  21...
Loss: 0.2317
Training epoch  22...
Loss: 0.0730
Training epoch  23...
Loss: 0.0245
Training epoch  24...
Loss: 0.0088
Training epoch  25...
Loss: 0.0046
Training epoch  26...
Loss: 0.0028
Training epoch  27...
Loss: 0.0019
Training epoch  28...
Loss: 0.

In [41]:
torch.save(net.state_dict(), 'trained_net.pth')

In [42]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [43]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = (correct/total)*100
print(f'Accuracy: {accuracy}%')

Accuracy: 100.0%


In [ ]:
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms

# Define transform to match training
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Function to load and transform image
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")  # ensure 3 channels
    image = new_transform(image)
    image = image.unsqueeze(0)  # add batch dimension
    return image

# Load all images in guess_images folder
guess_folder = Path("guess_images")
image_paths = list(guess_folder.glob("*.[jp][pn]g"))  # jpg/jpeg/png

# Predict
net.eval()
with torch.no_grad():
    for img_path in image_paths:
        image_tensor = load_image(img_path)
        output = net(image_tensor)
        _, predicted = torch.max(output, 1)
        print(f"{img_path.name} → Prediction: {class_names[predicted.item()]}")


b2.png → Prediction: Barack Obama
b1.png → Prediction: Barack Obama
